<a href="https://colab.research.google.com/github/Saultr21/IA-Y-BIGDATA/blob/main/PRO/QLEARNING_8_PUZZLE_BIEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CREA LA TABLA DE TRANSICIONES DEL 8 PUZZLE

### Explicación del código

Este código genera las posibles transiciones del juego 8-puzzle, representado como una cadena donde el número '9' simboliza el espacio vacío. Se estructura en varios componentes clave:

1. **Direcciones y movimientos válidos:** Se define una lista `directions` que indica los cambios de índice en la cadena para simular los movimientos del espacio vacío: arriba (-3), derecha (+1), abajo (+3) e izquierda (-1). La función `is_valid_move` verifica si un movimiento es válido según las reglas del tablero de 3x3, evitando que el espacio vacío se salga de los límites.

2. **Intercambio de posiciones:** La función `swap` toma un estado inicial como cadena, convierte este estado a una lista para modificarlo, intercambia dos posiciones indicadas y luego lo devuelve nuevamente como una cadena.

3. **Generación de transiciones:** La función principal `generate_transitions_8_puzzle` utiliza un enfoque de búsqueda en anchura (BFS) para explorar todos los estados posibles del juego desde un estado inicial dado. Usa una cola (`deque`) para procesar los estados y un conjunto (`visited`) para asegurarse de no procesar estados repetidos. Para cada estado, calcula las transiciones posibles al mover el espacio vacío en direcciones válidas, almacenando los nuevos estados generados en un diccionario `transitions`, donde las claves son los estados actuales y los valores son listas de estados resultantes.

4. **Inicialización:** Se define el estado inicial del tablero como la cadena `"123456789"` (el espacio vacío en la última posición), y se llama a la función para generar las transiciones desde este estado inicial.

El resultado final es un diccionario que mapea cada estado del juego con sus posibles movimientos, representando todas las configuraciones alcanzables desde el estado inicial.


In [ ]:
from collections import deque

# Define moves relative to the position of '9' (empty space)
# Each move corresponds to: up, right, down, left
directions = [-3, 1, 3, -1]  # Change in index for each move
#move_names = ["Up", "Right", "Down", "Left"]

# Validate if a move is within bounds
def is_valid_move(index, direction):
    if direction == -3 and index < 3:  # Up
        return False
    if direction == 3 and index > 5:  # Down
        return False
    if direction == -1 and index % 3 == 0:  # Left
        return False
    if direction == 1 and index % 3 == 2:  # Right
        return False
    return True

# Swap two positions in a state
def swap(state, i, j):
    state = list(state)
    state[i], state[j] = state[j], state[i]
    return ''.join(state)

# Generate transitions for the 8-puzzle
def generate_transitions_8_puzzle(initial_state):
    visited = set()
    queue = deque([initial_state])
    transitions = {}

    while queue:
        current_state = queue.popleft()
        if current_state in visited:
            continue
        visited.add(current_state)

        empty_index = current_state.index('9')
        state_transitions = []

        for direction in directions:
            if is_valid_move(empty_index, direction):
                new_state = swap(current_state, empty_index, empty_index + direction)
                state_transitions.append(new_state)
                if new_state not in visited:
                    queue.append(new_state)
            else:
                state_transitions.append(None)  # Invalid move

        transitions[current_state] = state_transitions

    return transitions

# Initial state of the puzzle
initial_state = "123456789"

# Generate the transitions
T = generate_transitions_8_puzzle(initial_state)


Por ejemplo T["123456789"]=['123459786', None, None, '123456798']
 y T["123456789"][1]=None

#TABLA DE RECOMPENSAS

### Explicación del código

La función `sequence_reward` calcula una recompensa basada en cuántos números del estado actual (`state`) coinciden en secuencia con el estado objetivo (`goal_state`), que representa la solución del 8-puzzle.

1. **Estado objetivo:** Se define como `"123456789"`, que es el orden correcto de los números con el espacio vacío ('9') al final.

2. **Secuencia correcta:**  
   - Se inicializa una cadena vacía `correct_sequence` para acumular los caracteres que coinciden con el estado objetivo desde el inicio.
   - Se recorre el estado actual (`state`) comparando carácter por carácter con el estado objetivo.
   - Si un carácter coincide, se agrega a `correct_sequence`. Si no coincide, el bucle se detiene con un `break`.

3. **Cálculo de la recompensa:**  
   - Si todos los números están en la posición correcta, la recompensa es `1000`.
   - Si no, la recompensa se calcula como `(len(correct_sequence) - 1) * 2`, que asigna un valor negativo proporcional al número de piezas consecutivas correctas menos uno. Esto penaliza estados que están lejos de la solución.


In [ ]:
def sequence_reward(state):
  goal_state = "123456789"  # Estado objetivo

  correct_sequence = ""  # Secuencia correcta acumulada

  # Construir la secuencia correcta desde el inicio del estado
  for i, char in enumerate(state):
    if char == goal_state[i]:
      correct_sequence += char
    else:
      break  # Detenerse en la primera pieza incorrecta

  # La recompensa será igual al número de piezas correctas en secuencia
  rewart=1000
  if len(correct_sequence)<9:
    rewart=(len(correct_sequence)-1)*2 # Recompensa base negativa
  return rewart


In [ ]:
R = {}
# Asignar recompensas basadas en la secuencia correcta
for state in T.keys():
  R[state] = sequence_reward(state)

In [ ]:
R["123496758"]

6

## INICIALIZACIÓN de la tabla Q a cero

### Explicación del código

La función `initialize_Q_table` se utiliza para crear e inicializar una tabla Q, que es una estructura clave en algoritmos de aprendizaje por refuerzo. La tabla Q almacena los valores de recompensa esperada para cada combinación de estado y acción.

1. **Parámetro de entrada:**  
   - `transitions`: Un diccionario donde las claves son los estados posibles del 8-puzzle y los valores son las listas de transiciones para cada acción.

2. **Inicialización de la tabla Q:**  
   - Se crea un diccionario vacío llamado `Q_table`.
   - Para cada estado en `transitions.keys()`, se añade una entrada en `Q_table` asociada a ese estado.
   - Cada estado tiene un diccionario interno que mapea las acciones posibles (en este caso, representadas como números del 0 al 3, correspondientes a los índices de las direcciones) a valores iniciales de `0`.

3. **Salida:**  
   - Devuelve `Q_table`, que contiene un valor inicial de 0 para cada estado y acción posible. Esto prepara la estructura para ser actualizada durante el entrenamiento del modelo de aprendizaje por refuerzo.


In [ ]:
# Función para inicializar la tabla Q a cero
def initialize_Q_table(transitions):
    Q_table = {}
    # Inicializar tabla Q a cero para cada estado y cada acción posible
    for state in transitions.keys():
        #Q_table[state] = {action: 0 for action in move_names}
        Q_table[state] = {action: 0 for action in range(4)}
    return Q_table #, df_Q

In [ ]:
Q=initialize_Q_table(T)

##Q-LEARNING
Ejemplo de Estado aleatorio seleccionado: 621849375
Transiciones válidas desde este estado: ['629841375', None, '621845379', '621894375'].
T['621849375'][2]='621845379'
Q['123456789'] vale inicialmente {0: 0, 1: 0, 2: 0, 3: 0}

## Q-learnning sin épsilon greedy

### Explicación del código

Este código implementa el algoritmo Q-learning para entrenar un agente en el entorno del 8-puzzle, utilizando la tabla Q para aprender las mejores acciones en cada estado con base en las recompensas y transiciones.

#### **1. Hiperparámetros:**
- `v` (tasa de aprendizaje): Determina cuánto impacto tiene el nuevo conocimiento sobre el valor existente en la tabla Q.
- `y` (factor de descuento): Pondera la importancia de las recompensas futuras frente a las recompensas inmediatas.

#### **2. Selección del estado inicial:**
- Los estados posibles se obtienen de las claves del diccionario de transiciones `T`.
- Se selecciona un estado inicial aleatorio usando `np.random.choice`.

#### **3. Bucle de entrenamiento:**
El bucle principal entrena el modelo durante un número definido de episodios (`max_entrenamientos`):
- **Selección de acción:**  
  Se elige una acción aleatoria (0 a 3) y se valida que la transición sea válida (no `None`). Si no lo es, se elige otra acción.
- **Estado siguiente:**  
  La acción seleccionada lleva a un nuevo estado según el diccionario de transiciones `T`.
- **Actualización de la tabla Q:**  
  Utiliza la **ecuación de Bellman**:
  \[
  Q(s, a) = (1 - v) \cdot Q(s, a) + v \cdot (R(s) + y \cdot \max(Q(s', \text{todas las acciones})))
  \]
  - `R(s)` es la recompensa asociada al estado actual.
  - `Q(s', todas las acciones)` evalúa el máximo valor Q posible desde el estado siguiente.
- **Manejo del estado objetivo:**  
  Si el estado objetivo (`"123456789"`) se alcanza, se reinicia el agente en un estado aleatorio.
- **Visualización periódica:**  
  Cada 1,000,000 de episodios, imprime el valor Q actual del estado.

#### **4. Finalización:**
El entrenamiento termina cuando se completan todos los episodios, dejando una tabla Q que guía al agente hacia el estado objetivo con recompensas máximas.



In [ ]:
import numpy as np
#HIPERPARÁMETROS
v = 0.1  # Factor de actualización (también conocido como tasa de aprendizaje)
y = 0.9  # Factor de descuento
# SELECCIÓN DEL ESTADO INICIAL
# Listar todos los estados posibles
estados = list(T.keys())
# Seleccionar un estado aleatorio de los posibles estados
s = np.random.choice(estados) # ESTADO INICIAL
entrenar = 0
max_entrenamientos = 10000000  # Número de episodios de entrenamiento
while entrenar < max_entrenamientos:
    # Acción aleatoria al azar (elige un número entre 0 y 3)
    a = np.random.randint(4)
    # Asegurarse de que la transición es válida
    while T[s][a] == None:
        a = np.random.randint(4)  # Reintentar si la acción no es válida
    # Estado siguiente
    siguiente = T[s][a]
    # Actualizar la tabla Q con la fórmula de Q-learning
    Q[s][a] = (1 - v) * Q[s][a] + v * (R[s] + y * max(Q[siguiente].values())) # ECUACIÓN DE BELLMAN
    # Mostrar el estado y el siguiente estado
    #print(f"{s} --> {siguiente}")
    if entrenar%1000000==0:
      print(Q[s])
    # Actualizar el estado actual
    if siguiente != "123456789":
        s = siguiente  # Estado siguiente
    else:
        # Si llegamos al estado 26, volvemos a un estado al azar
        s=np.random.choice(list(T.keys()))
    # Incrementar el contador de episodios
    entrenar += 1


{0: -0.2, 1: 0, 2: 0, 3: 0}
{0: 0, 1: -0.542, 2: 0, 3: -0.38}
{0: 0, 1: -0.38, 2: -0.485804, 3: -0.38}
{0: -1.0434062000000002, 1: -1.2251590220000002, 2: -0.8190200000000001, 3: -1.5424641509007802}
{0: -0.6878000000000001, 1: -1.0434062000000002, 2: -1.0434062000000002, 3: -0.8190200000000001}
{0: -1.2251590220000002, 1: 0, 2: 0, 3: -1.0434062000000002}
{0: 0, 1: -0.8190200000000001, 2: -1.188722, 3: -0.8190200000000001}
{0: -1.6998107294060019, 1: -1.7568466908188616, 2: -1.895330473394528, 3: -1.7568466908188616}
{0: -1.7811620217369755, 1: -1.6998107294060019, 2: -1.6998107294060019, 3: -1.37237880782}
{0: -1.629395962229632, 1: -1.8564204024616295, 2: -1.4916268343342, 3: -1.629395962229632}


## Q-learnning con épsilon greedy

### Explicación del código

Este código implementa un algoritmo de Q-learning con una estrategia **epsilon-greedy**, que alterna entre exploración y explotación para entrenar al agente en el entorno del 8-puzzle.

#### **1. Hiperparámetros:**
- `v` (tasa de aprendizaje): Controla cuánto afecta el nuevo conocimiento sobre el valor Q actual.
- `y` (factor de descuento): Pondera la importancia de las recompensas futuras.
- `epsilon`: Probabilidad inicial de explorar acciones aleatorias.
- `epsilon_min`: Valor mínimo al que puede reducirse `epsilon`, garantizando un mínimo de exploración.
- `epsilon_decay`: Factor por el cual `epsilon` disminuye en cada episodio, favoreciendo la explotación a medida que el agente aprende.

#### **2. Selección del estado inicial:**
- Se elige un estado aleatorio (`s`) de entre las claves del diccionario de transiciones `T`.

#### **3. Bucle de entrenamiento:**
El entrenamiento se realiza durante un número definido de episodios (`max_entrenamientos`):

- **Filtrar acciones válidas:**  
  Se identifican las acciones posibles desde el estado actual (`s`) que conducen a estados válidos en `T`.

- **Selección de acción (Epsilon-greedy):**  
  - Con probabilidad `epsilon`, el agente explora seleccionando una acción válida al azar.
  - Con probabilidad `1 - epsilon`, el agente explota, eligiendo la acción con el mayor valor Q.

- **Transición de estado:**  
  El agente avanza al siguiente estado (`siguiente`) según la acción seleccionada.

- **Actualización de la tabla Q:**  
  Utiliza la ecuación de Bellman:
  \[
  Q(s, a) = (1 - v) \cdot Q(s, a) + v \cdot (R(s) + y \cdot \max(Q(s', \text{todas las acciones})))
  \]
  Donde:
  - `R(s)` es la recompensa del estado actual.
  - `max(Q(siguiente).values())` es el valor máximo de Q del estado siguiente.

- **Reinicio al alcanzar el estado objetivo:**  
  Si se llega al estado objetivo (`"123456789"`), el agente reinicia desde un estado aleatorio.

- **Reducción de `epsilon`:**  
  `epsilon` se reduce en cada episodio multiplicándolo por `epsilon_decay`, hasta alcanzar el límite inferior (`epsilon_min`).

#### **4. Progreso y finalización:**
- Cada 100,000 episodios, muestra el progreso, incluyendo el valor actual de `epsilon` y el estado de la tabla Q.
- El entrenamiento termina cuando se alcanzan `max_entrenamientos` episodios.

**Ventajas del enfoque:**
- La estrategia epsilon-greedy balancea exploración y explotación.
- La reducción gradual de `epsilon` asegura que el modelo priorice la explotación al final del entrenamiento.


In [ ]:
import numpy as np

# HIPERPARÁMETROS
v = 0.7  # Tasa de aprendizaje
y = 0.8  # Factor de descuento
epsilon = 0.95  # Probabilidad de exploración inicial
epsilon_min = 0.01  # Epsilon mínimo
epsilon_decay = 0.99999995  # Factor de decaimiento de epsilon

# SELECCIÓN DEL ESTADO INICIAL
estados = list(T.keys())
s = np.random.choice(estados)
entrenar = 0
max_entrenamientos = 10000000

while entrenar < max_entrenamientos:
    # Filtrar acciones válidas
    acciones_validas = [a for a in Q[s] if T[s][a] is not None]

    if not acciones_validas:
        print(f"No hay acciones válidas desde el estado {s}")
        break  # Si no hay acciones válidas, salir del ciclo

    # Seleccionar acción usando epsilon-greedy
    if np.random.rand() < epsilon:
        # Exploración: elegir una acción válida al azar
        a = np.random.choice(acciones_validas)
    else:
        # Explotación: elegir la mejor acción válida
        a = max(acciones_validas, key=lambda a: Q[s][a])

    # Obtener el siguiente estado
    siguiente = T[s][a]

    # Actualizar la tabla Q usando la ecuación de Bellman
    Q[s][a] = (1 - v) * Q[s][a] + v * (R[s] + y * max(Q[siguiente].values()))

    # Mostrar progreso cada 10,000 episodios
    if entrenar % 100000 == 0:
        print(f"Episodio {entrenar}, Epsilon: {epsilon:.4f}")
        print(Q[s])

    # Actualizar el estado actual
    if siguiente != "123456789":
        s = siguiente
    else:
        # Si llegamos al estado objetivo, reiniciar a un estado aleatorio
        s = np.random.choice(estados)

    # Reducir epsilon para priorizar explotación a medida que se aprende
    epsilon = max(epsilon_min, epsilon * epsilon_decay)

    # Incrementar el contador de episodios
    entrenar += 1



Episodio 0, Epsilon: 0.9500
{0: 0, 1: -1.5882177358107021, 2: -1.9270540072456583, 3: 0}
Episodio 100000, Epsilon: 0.9453
{0: -1.9629395962229632, 1: -1.9840467113846256, 2: 0, 3: 0}
Episodio 200000, Epsilon: 0.9405
{0: 0.0, 1: 0.23527208122416535, 2: 0.0900801209604714, 3: 0}
Episodio 300000, Epsilon: 0.9359
{0: -1.9343486065210926, 1: 0, 2: -1.999905797426055, 3: -3.27929217717932}
Episodio 400000, Epsilon: 0.9312
{0: -1.6998107294060019, 1: 0, 2: -1.9099432188218004, 3: -2.49390850607122}
Episodio 500000, Epsilon: 0.9265
{0: -3.233153749426748, 1: -1.9270540072456583, 2: 0, 3: -1.9468223712820851}
Episodio 600000, Epsilon: 0.9219
{0: 0, 1: 0, 2: 6.157383612622772, 3: 3.1972338167689447}
Episodio 700000, Epsilon: 0.9173
{0: -1.8305422781114, 1: -1.9189488969396205, 2: -1.996123350866464, 3: -1.9343486065210926}
Episodio 800000, Epsilon: 0.9127
{0: -1.9840467113846256, 1: -3.5584298086186976, 2: -1.999905797426055, 3: 0}
Episodio 900000, Epsilon: 0.9082
{0: 0, 1: -1.9992369591510462, 

## SOLUCIÓN

In [ ]:
import numpy as np
path = []
# Mostrar los pasos para resolver el 8-puzzle desde un estado inicial aleatorio
def resolver_8_puzzle(T, Q, estado_inicial, estado_objetivo="123456789"):
    # Estado inicial aleatorio si no se proporciona uno
    if estado_inicial is None:
        estado_inicial = np.random.choice(list(T.keys()))
    print(f"Estado inicial: {estado_inicial}")

    # Recorrer los pasos
    estado_actual = estado_inicial
    pasos = 0

    while estado_actual != estado_objetivo:
        #print(f"Paso {pasos}: {estado_actual}")
        print(estado_actual)
        path.append(estado_actual)
        # Elegir la mejor acción basada en la tabla Q
        acciones = Q[estado_actual]
        # Filtrar acciones válidas
        acciones_validas = [a for a in Q[estado_actual] if T[estado_actual][a] is not None]

        accion_mejor = max(acciones_validas, key=acciones.get)

        # Aplicar la acción para obtener el siguiente estado
        siguiente_estado = T[estado_actual][accion_mejor]

        # Verificar que el siguiente estado sea válido
        if siguiente_estado is None:
            print(f"Error: no hay una transición válida desde el estado {estado_actual} con la acción {accion_mejor}.")
            break

        # Actualizar el estado actual
        estado_actual = siguiente_estado
        pasos += 1

        # Salida para verificar el estado
        if pasos > 100:  # Prevenir loops infinitos
            print("El algoritmo no pudo resolver el puzzle en un número razonable de pasos.")
            break

    # Imprimir el estado final
    if estado_actual == estado_objetivo:
        print(f"¡Puzzle resuelto en {pasos} pasos! Estado final: {estado_actual}")
    else:
        print("No se pudo resolver el puzzle.")

# Ejemplo de uso
estado_inicial = None  # Deja como None para seleccionar un estado aleatorio
resolver_8_puzzle(T, Q, estado_inicial)


Estado inicial: 957346182
957346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
597346182
579346182
5973

In [ ]:
Q["541872369"]


{0: -1.9999999954943202, 1: 0, 2: 0, 3: -1.9999999915217683}

In [ ]:
T["541872369"]

['541879362', None, None, '541872396']

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.pyplot as plt

def animate_puzzle_solution(path):
    """
    Muestra la animación del puzzle 8 desde el estado inicial al estado objetivo.

    path: Lista de estados que representan el camino desde el estado inicial al objetivo.
    """
    # Crear la figura
    fig, ax = plt.subplots()
    ax.axis("off")
    tiles = [[None for _ in range(3)] for _ in range(3)]

    # Inicialización del gráfico
    def init():
        for i in range(3):
            for j in range(3):
                tiles[i][j] = ax.text(
                    j, -i, "", ha="center", va="center", fontsize=20,
                    bbox=dict(boxstyle="round", facecolor="white", edgecolor="black")
                )
        ax.set_xlim(-0.5, 2.5)
        ax.set_ylim(-2.5, 0.5)
        return [tile for row in tiles for tile in row]

    # Actualización para cada frame
    def update(frame):
        state = path[frame]
        for i, value in enumerate(state):
            x, y = i % 3, i // 3
            tiles[y][x].set_text("" if value == '9' else value)
        return [tile for row in tiles for tile in row]

    # Crear la animación
    anim = FuncAnimation(fig, update, frames=len(path), init_func=init, blit=True, repeat=False)
    plt.close(fig)
    return HTML(anim.to_jshtml())

# ============================
# EJEMPLO DE USO
# ============================

# Simulación de un camino de solución (puedes usar tu función find_path_to_goal para generarlo)

# Mostrar la animación
animate_html = animate_puzzle_solution(path)
display(animate_html)